A recent family game of quarantine Yahtzee disintegrated into an argument over how many turns it would take you to get a yahtzee. Soon, my little brother, who had just learned about object oriented programming, wrote a Yahtzee class to answer the question; my dad, who's way too smart, worked out the probabilities by hand, while my partner rolled physical dice and counted the number of yahtzees with ticks on a sheet of paper.  

I took what I consider to be the lazy route and answered the question with Monte Carlo sampling and basic code. Monte Carlo sampling uses large numbers of random trials to calculate results that might be tedious or too difficult calculate by hand (it took my dad an hour to work out the yahtzee probabilities by hand). Bayesians may recognize Monte Carlo sampling as a great way to calculate posterior distributions; history buffs may recognize it as a key step that led to the development of the atomic bomb. 

To use Monte Carlo sampling to figure out how many turns it would take you to roll a yahtzee, we first need to find out the probability of rolling a yahtzee on any given turn.    
  
We define a turn as three total rolls, and since we're methodical Yahtzee players who rely on a plan rather than luck, we have decision rules for how to approach our rolls: 
  * for any roll, if there are two or more dice with the same number, we keep those dice and re-roll the rest
  * if there are two dice with one number and two dice with another, we randomly choose between them to decide which to keep
  


In [1]:
import numpy as np
from collections import Counter
import random

In [2]:
def roll_dice(previous_roll):
    '''
    Roll any dice that we haven't decided to keep. 
    
    Args:
        previous_roll, np.array: array containing the dice, if any, that
        we have decided to keep from the previous roll. 
    
    Returns:
        np.array of size 5 containing the numbers on the dice we currently
        have.
    '''
    roll_size = 5 - len(previous_roll)
    new_roll = np.random.randint(1, 7, roll_size)
    return np.append(previous_roll, new_roll)

def find_what_to_keep(roll_list):
    '''
    Calculate the mode 
    '''
    count = 0
    mode_list = []
    counted = Counter(roll_list).most_common()
    for die in counted:
        if die[1] >= count:
            count = die[1]
            mode_list.append(die[0])
    
    return mode_list, count  

Let's see what happens when we `play_one_round_of_yahtzee`: 

In [3]:
def play_one_round_of_yahtzee():
    previous_roll = np.array([])
    roll_count = 0
    mode_count = 0
    while roll_count < 3 and mode_count < 5:
        roll = roll_dice(previous_roll)
        print(roll)
        to_keep = find_what_to_keep(roll)
        mode_count = to_keep[1]
        if mode_count > 1:
            keep = random.choice(to_keep[0])
            previous_roll = roll[np.where(roll == keep)]
        roll_count += 1
    if mode_count == 5:
        return 1
    else:
        return 0

play_one_round_of_yahtzee()

[2. 4. 5. 3. 1.]
[1. 2. 3. 4. 1.]
[1. 1. 6. 5. 1.]


0

In the first round, we roll a full straight (five numbers in a row), but we're only playing for yahtzees, so we throw it away. In the second round, we roll two 1's, so we keep them. In the third round, we roll one more 1 and keep it, but it's still not good enough for a yahtzee, so we return 0 to indicate a failure.

Let's redefine the function without the print statement to keep the output sane, then try some basic Monte Carlo sampling. 

In [4]:
def play_one_round_of_yahtzee():
    previous_roll = np.array([])
    roll_count = 0
    mode_count = 0
    while roll_count < 3 and mode_count < 5:
        roll = roll_dice(previous_roll)
        to_keep = find_what_to_keep(roll)
        mode_count = to_keep[1]
        if mode_count > 1:
            keep = random.choice(to_keep[0])
            previous_roll = roll[np.where(roll == keep)]
        roll_count += 1
    if mode_count == 5:
        return 1
    else:
        return 0

In [5]:
yahtzees = []
for i in range(1000):
    yahtzees.append(play_one_round_of_yahtzee())
np.mean(yahtzees)

0.056

It looks like slightly over 5% of turns can be expected to yield a yahtzee. However, this is with a small number of samples. The army, as you might imagine, cares about Monte Carlo sampling, and the Army Research Lab published a [report](https://apps.dtic.mil/dtic/tr/fulltext/u2/a621501.pdf) encouraging people to make better choices with their Monte Carlo sampling schemes. 

The important takeaway for us from this report is that the accuracy of Monte Carlo is proportional to $1/\sqrt(n)$, where n is the number of simulations. In order to get three decimals of accuracy, we need to run about a million simulations. 

This may take a little while, as Python is probably not the best language to run this simulation. I've recently started learning Stan, which is a) awesome b) named after Stanislaw Ulam, the guy who invented the modern version of Monte Carlo sampling at the Los Alamos National Laboratory in the 1940s, and c) probably able to make short work of this problem in a matter of seconds. I may revisit this in the future with a Stan refactor. 

In [6]:
%%time

yahtzees = []
for i in range(1000000):
    yahtzees.append(play_one_round_of_yahtzee())
np.mean(yahtzees)

CPU times: user 2min 37s, sys: 1.99 s, total: 2min 39s
Wall time: 2min 41s


0.046256

It looks like the probability of getting a yahtzee on a single turn is about 4.6%. So how many turns can you expect to have before you roll a yahtzee, assuming that's all you focus on? The [geometric distribution](https://en.wikipedia.org/wiki/geometric_distribution) calculates the probability of _k_ trials being necessary to ensure one success. 

Thanks to the baseball cards full of info Wikipedia helpfully supplies for basically every probability distribution, we know that the expected value of the geometric distribution - in other words, the amount of turns we'd expect to need before we get a yahtzee - is given by $\frac{1}{p}$, where _p_ is the probability of success. 

In [7]:
1 / 0.046

21.73913043478261

It looks like we'd need about 22 turns to get a yahtzee. Now you know why most yahtzee players can go whole games without getting a yahtzee - this simulation shows 22 turns of *solely trying to get a yahtzee*. If you were to mix in attempts to get all the other kinds of rolls, you would follow different decision rules than the ones we laid out for getting a yahtzee, and it would probably take you even longer to get one. 

As it happens, my partner and I rolled dice and counted the number of rolls for about as long as we could stand (four yahtzees), and it generally took us around 20 rolls to succeed. My Monte Carlo result therefore has some extremely insignificant real-world experimental evidence to back it up. If we stay in quarantine long enough, we may end up like [the statistician in the concentration camp who flipped a coin 10,000 times](https://en.wikipedia.org/wiki/John_Edmund_Kerrich#Experiments_in_empirical_probability) and start generating some real experimental evidence.